# Homework Assignment 2: Data Prep
In this homework assignment, you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This assignment will have you explore the cardinalities, number of missing values, detect outliers, handle missing values and outliers, and create data quality report for original and cleaned dataset. Additionally, you will be asked to provide documentation for your functions.

## Step 1: Downloading the Data

This assignment will only be using [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB). Recall that in Homework 1 we started to construct the analytics base table for our [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM). In that assignment, we read the data from the two subdirectories, __FL__ and __NF__, of the __partition1__ direcotry. These two subdirectories represented the two classes of our target feature in the solar flare prediction problem we will be attempting to solve this semester. We then processed these samples of multivariate time series to construct descriptive features for each sample, and then placed them into our analytics base table.

In this assignment, you will be utilizing a set of extracted descriptive features much like what you were asked to construct in Homework 1. However, this dataset contains many more extracted features than you were asked to compute for Homework 1 (>800), so we need to explore the data to find data quality issues and identify ways to address these issues. Below are links to the full extracted feature dataset for partition 1 and a toy dataset to use for testing you functions.

__Note:__ Since the full dataset, and multiple copies of partially processed intermediary results, tend to take up a bit of space, you can use the toy dataset to implement and test your code. You may need to edit the data to fully test each of the requirements, but that is left as an exercise for the student. The full partition dataset is only included for those who wish to work with it once they have their code implemented. 

- [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv)
- [Toy Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_partition1ExtractedFeatures.csv)

Now that you have the extracted features csv files, you will load that data into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method.  


In [1]:
import os
import pandas as pd
from pandas import DataFrame 
import numpy as np
from IPython.display import display

### Read the dataset

In [2]:
data_dir = r'C:\Users\Tracy\DSWorkspace\hw2\data\MVTS'
# toy
# data_file = 'toy_partition1ExtractedFeatures.csv'
#full set
data_file = 'partition1ExtractedFeatures.csv'

In [3]:
abt = pd.read_csv(os.path.join(data_dir, data_file), index_col=0)

In [4]:
# making sure it loaded
abt.head()

,id,lab,st,et,TOTUSJH_min,TOTUSJH_max,TOTUSJH_median,TOTUSJH_mean,TOTUSJH_stddev,TOTUSJH_var,...,R_VALUE_gderivative_kurtosis,R_VALUE_linear_weighted_average,R_VALUE_quadratic_weighted_average,R_VALUE_average_absolute_change,R_VALUE_average_absolute_derivative_change,R_VALUE_last_value,R_VALUE_slope_of_longest_mono_increase,R_VALUE_slope_of_longest_mono_decrease,R_VALUE_avg_mono_increase_slope,R_VALUE_avg_mono_decrease_slope
0,514,C,2011-04-28T23:12:00,2011-04-29T11:00:00,1654.553362,1815.727348,1725.894296,1732.208184,44.055204,1940.861005,...,-0.810154,4.478583,4.496122,0.024698,0.038708,4.574968,0.032692,-0.000901,0.016224,-0.014214
1,107,NF,2010-08-04T06:36:00,2010-08-04T18:24:00,38.257743,104.732191,48.946497,54.105280,15.635504,244.468972,...,21.885734,1.288584,1.696440,0.084950,0.141598,2.817515,0.704938,-0.053054,0.242427,-0.090127
2,798,C,2011-08-21T05:12:00,2011-08-21T17:00:00,807.927312,927.994392,860.664804,861.078366,35.013665,1225.956749,...,2.698058,4.084092,4.132907,0.052272,0.081651,4.201446,0.110665,-0.004080,0.032708,-0.031965
3,1372,NF,2012-02-02T11:48:00,2012-02-02T23:36:00,97.702918,133.588624,110.826583,112.290552,8.399740,70.555625,...,-3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,475,NF,2011-04-09T01:24:00,2011-04-09T13:12:00,287.054395,357.415061,317.865356,321.454795,14.461723,209.141437,...,-3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Q1 (20 points)

Write a function to extract the various pieces of a data quality report, for a specific attribute, and return a dataframe with this information.

 * 'Feature Name': Contains the time series statistical feature name
 
 * 'Cardinality': Contains the count of unique values for the feature
            
 * 'Non-null Count': Contains the number of non-null entries for the feature
            
 * 'Null Count': Contains the number of null or missing entries for the feature
            
 * 'Min': Contains the minimum value of the feature (Without considering the null or nan value)
 
 * '25th': Contains the first quartile (25%) of the feature values (Without considering the null/nan value)
 
 * 'Mean': Contains the mean of the feature values (Without considering the null/nan value)
 
 * '50th': Contains the median of the feature values (Without considering the null/nan value)
            
 * '75th': Contains the third quartile (75%) of the feature values (Without considering the null/nan value)
 
 * 'Max': Contains the maximum value of the feature (Without considering the null/nan value),
            
 * 'Std. Dev': Contains the standard deviation of the feature (Without considering the null/nan value)
 
In addition to the values above, you should identify the number of upper and lower outliers using the $val < Q1-1.5IQR$ and $val > Q3+1.5IQR$ outlier identification method. These added features should be called `Outlier Count Low` and `Outliers Count High` respectively.


 
 Some useful functions for this can be found at:
 
 * [Numpy.percentile](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html)
 
 * [pandas.isna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isna.html)
 
 * [Numpy.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)
 
 * [Numpy.std](https://numpy.org/doc/stable/reference/generated/numpy.std.html)
 

In [5]:
def calc_summary_for(feature_name:str, data:DataFrame) -> DataFrame:
    """
    Calculate a summary of a specified feature from a given DataFrame
    Items to be calculated (becomes column names of output):
        'Feature Name': Contains the time series statistical feature name
        'Cardinality': Contains the count of unique values for the feature
        'Non-null Count': Contains the number of non-null entries for the feature
        'Null Count': Contains the number of null or missing entries for the feature
        'Min': Contains the minimum value of the feature (Without considering the null or nan value)
        '25th': Contains the first quartile (25%) of the feature values (Without considering the null/nan value)
        'Mean': Contains the mean of the feature values (Without considering the null/nan value)
        '50th': Contains the median of the feature values (Without considering the null/nan value)
        '75th': Contains the third quartile (75%) of the feature values (Without considering the null/nan value)
        'Max': Contains the maximum value of the feature (Without considering the null/nan value),
        'Std. Dev': Contains the standard deviation of the feature (Without considering the null/nan value)
        
    Parameters
    -----------
    feature_name : str
        name of the feature on which summary will be calculated
        feature name must be present in passed DataFeame
    
    data : DataFrame
        a DataFrame containing features and data to be calculated
            
    Returns
    -------
    DataFrame
        DataFrame containing columns with summary feature names and a row with calculated values for each column
    """
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Std. Dev','Outlier Count Low', 'Outlier Count High']
    # Your answer to Q1 goes here!
    # Make sure to return a DataFrame with the features specified.
    
    result_ls = []
    
    # 5 num summary
    min = data[feature_name].min()
    max = data[feature_name].max()
    quarts = np.percentile(data[feature_name].to_numpy(), [25, 50, 75], interpolation='midpoint')
    
    # num outliers
    iqr = quarts[2] - quarts[0]
    iqr_e = 1.5 * iqr
    out_min_ct = len([i for i in data[feature_name] if i < (quarts[0] - iqr_e)])
    out_max_ct = len([i for i in data[feature_name] if i > (quarts[2] + iqr_e)])
    
    # construct summary
    # name
    result_ls.append(feature_name)
    # card
    result_ls.append(len(data[feature_name].unique()))
    # non-null ct
    result_ls.append(data[feature_name].notna().sum())
    # is null ct
    result_ls.append(data[feature_name].isna().sum())
    # min
    result_ls.append(min)
    # 25th
    result_ls.append(quarts[0])
    # mean
    result_ls.append(np.mean(data[feature_name].to_numpy()))
    # 50th
    result_ls.append(quarts[1])
    # 75th
    result_ls.append(quarts[2])
    # max
    result_ls.append(max)
    #std dev
    result_ls.append(np.std(data[feature_name].to_numpy()))
    # out low ct
    result_ls.append(out_min_ct)
    # out high ct
    result_ls.append(out_max_ct)
    
    
    # return DataFrame of calc summary
    return pd.DataFrame([result_ls], columns = summary_feature_names)    
    pass

In [6]:
summary_TOTUSJH_min = calc_summary_for('TOTUSJH_min', abt)
# check output
display(summary_TOTUSJH_min)

,Feature Name,Cardinality,Non-null Count,Null Count,Min,25th,Mean,50th,75th,Max,Std. Dev,Outlier Count Low,Outlier Count High
0,TOTUSJH_min,30165,73492,0,0.0,29.75602,396.697968,110.011893,418.614528,5680.232811,694.293786,0,9014


### Q2 (20 points)
Using what you produced to answere Q1, you should now write a function to construct the data quality report for all of the numerical features of our dataset.  You should loop over all of the features in the analytics base table represented by the input feature dataset files from partition 1, with the exception of the first column (this is the index column if you read the file correctly), and the `id`, `lab`, `st`, and `et` columns.  

Your output from this function will be a DataFrame that has 1 row for each feature. 

In [7]:
def construct_quality_report(data:DataFrame) -> DataFrame:
    """
    calculates the summary for each feature in a DataFrame using the calc_summary_for() function
    Excludes features ['id', 'lab', 'st', 'et']
    
    Parameters
    ----------
    data : DataFrame
        a DataFrame containing features and data to calculate a summary
        
    Returns
    -------
    DataFrame
        a DataFrame containing columns of calculated summary items and rows of features from input data
    
    """
    
    excluded_columns = ['id', 'lab', 'st', 'et']
    
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Std. Dev','Outlier Count Low', 'Outlier Count High']
    
    # Your answer to Q2 goes here!
    # Make sure to return a DataFrame with the features specified.    
    result_ls = [calc_summary_for(i, data).values.flatten().tolist() for i in data.columns if i not in excluded_columns]
    return pd.DataFrame(result_ls, columns = summary_feature_names)

    pass

In [8]:
summary_table = construct_quality_report(abt)
print(summary_table.shape)  # checking the dimensionality is often a useful test.
summary_table.head()

(816, 13)


,Feature Name,Cardinality,Non-null Count,Null Count,Min,25th,Mean,50th,75th,Max,Std. Dev,Outlier Count Low,Outlier Count High
0,TOTUSJH_min,30165,73492,0,0.000000,29.756020,396.697968,110.011893,418.614528,5680.232811,694.293786,0,9014
1,TOTUSJH_max,24142,73492,0,0.783099,53.115131,492.649021,162.797367,541.875882,6078.237599,801.078303,0,8540
2,TOTUSJH_median,62001,73492,0,0.000000,40.726051,439.417610,133.981693,472.725349,5895.663839,741.152887,0,8799
3,TOTUSJH_mean,73492,73492,0,0.100756,40.832741,440.333385,134.524140,474.264394,5877.701432,741.767409,0,8783
4,TOTUSJH_stddev,73492,73492,0,0.176471,4.926582,25.019193,11.410790,30.408896,618.834925,35.130481,0,6684


### Q3 (20 points)
#### Drop features with improper cardinality:
Using the quality report summary table that is returned from the function you wrote for Q2, we are now going to investigate our data. For this, you should use the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Since we are using real valued features, a majority of them shall have a cardinality close to the sample count. So, for this question, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the feature that have a cardinality less than 10. This feature should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

A useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [9]:
def drop_low_card_data(summary_table:DataFrame, data:DataFrame) -> None:
    # Your answer to Q3 goes here! 
    """
    Drop features from DataFrame that have a cardinality less than 10
    Drops are performed in place so returns None
    
    Parameters
    ----------
    summary_table : DataFrame
        DataFrame containing summary values of a dataset
        must contain 'Cardinality' and 'Feature Name' features
    
    data : DataFrame
        input dataset, an analytics base table for which summary_table was calculated
        
    Returns
    -------
        None
    """    
       
    # TODO - remove
    # while this does work, it also drops the features that were excluded in previous question from the abt
    # yeilds shape summary_table.shape == (801, 13) and len(abt.columns) == 801 (should be 805)
#     summary_table.drop([i for i in summary_table.index if summary_table['Cardinality'][i] < 10], inplace=True)
#     data.drop(list(set(data.columns.tolist()) - set(summary_table['Feature Name'].tolist())), 1, inplace=True)

    # TODO - remove          
    # yeilds shape summary_table.shape == (816, 13) and len(abt.columns) == 820 (???????????)
    # not sure why there are 15 extra features...
    # also this is just messy anyway, using append in list comp
#     drop_i = []
#     drop_n = []
#     (drop_i.append(i) and drop_n.append(summary_table['Feature Name'][i])\
#     for i in summary_table.index if summary_table['Cardinality'][i] < 10)

#     data.drop(drop_n, inplace=True)    
#     summary_table.drop(drop_i, inplace=True)        


    # yeilds shape summary_table.shape == (801, 13) and len(abt.columns) == 805 (expected)
    for i in summary_table.index:
        if summary_table['Cardinality'][i] < 10:
            # order of drops is important here***********
            data.drop(summary_table['Feature Name'][i],1, inplace=True)
            summary_table.drop(i, inplace=True)    
    
    pass

In [10]:
drop_low_card_data(summary_table, abt)
print(summary_table.shape)
#print(len(abt.columns))

(801, 13)


### Q4 (20 points)
#### Drop features with excessive NaN

Again, using the quality report summary table that is returned from the function you wrote for Q2, we are going to continue investigating our data. For this, you should still be using the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Like the features that were dropped for Q3, some of the extracted features don't work on all of the variates of the input multi-variate time series samples very well.  So, some of these features return an excessive number of `NaN` values.  These are not verry useful features, so we want to get rid of them before we continue. To do this, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the features that have **more than 1%** of the entries as null/nan values. Again, these features should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

As in Q3, a useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [11]:
def drop_excessive_nan_data(summary_table:DataFrame, data:DataFrame) -> None:
    # Your answer to Q4 goes here!
    """
    Drops features from a DataFrame in which more than 1% of its entries are null/nan values
    Drops are performed in place so returns None
    
    Parameters
    ----------
    summary_table : DataFrame
        DataFrame containing summary values of a dataset
        must contain 'Null Count', 'Non-null Count' and 'Feature Name' features
    
    data : DataFrame
        input dataset, an analytics base table for which summary_table was calculated
        
    Returns
    -------
        None
    """   
    
    for i in summary_table.index:
        if summary_table['Null Count'][i] / summary_table['Non-null Count'][i] > 0.01:
            # order matters here****************
            data.drop(summary_table['Feature Name'][i],1, inplace=True)
            summary_table.drop(i, inplace=True)            
    pass

In [12]:
drop_excessive_nan_data(summary_table, abt)
print(summary_table.shape)
#print(len(abt.columns))

(799, 13)


Now that you have the data cleaned up a little, save the results of both your summary table and your analytics base table using the [pandas.to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) method. We will want to use these results for the next homework assignment.

In [13]:
out_dir = r'C:\Users\Tracy\DSWorkspace\hw2\data\MVTS'
out_summary_table_name = 'data_quality_table.csv'
out_abt_name = 'cleaned_partition1ExtractedFeatures.csv'

In [14]:
summary_table.to_csv(os.path.join(out_dir, out_summary_table_name))
abt.to_csv(os.path.join(out_dir, out_abt_name))

### Q5 (20 points)
#### Add docstring to your functions

Let's revisit our programming skill while learning Fundamentals of Data Science. 

Your code is only as valuable as its reusability. Without understandable and legible documentation (which makes maintainability and reusability possible) nobody would like to use your code, let alone to pay for it. ;)

If you want to know more about the value of documentation, read [this article](https://www.freecodecamp.org/news/why-documentation-matters-and-why-you-should-include-it-in-your-code-41ef62dd5c2f/). There are even conferences on this topic; see [this website](https://www.writethedocs.org/guide/writing/beginners-guide-to-docs/).

In Python, the documentation that is embedded in the code is called **docstring**. In the example below, the "string" wrapped in triple quotes is there to tell us all about this function.

In [15]:
def nanmean(a, axis=None):
    """
    Compute the arithmetic mean along the specified axis, ignoring NaNs.
    
    Parameters
    ----------
    a : array_like
        Array containing numbers whose mean is desired. If `a` is not an
        array, a conversion is attempted.
    axis : {int, tuple of int, None}, optional
        Axis or axes along which the means are computed. The default is to compute
        the mean of the flattened array.
    """
    # some magic happens here that we don't care about.
    pass

Note that this is not just a *comment*. If you execute `nanmean` and then call it (as if you want to use it), you can hit `shift+Tab` while your cursor is on the function name, and see how the docstring gets compiled and then pops up. This allows other users to see our description even when they don't have access to our source code. Try it! You can do this with other NumPy and Pandas functions/methods that you've been using.

The above example is a simplified version of the method `nanmean` copied from the NumPy library ([here](https://github.com/numpy/numpy/blob/v1.21.0/numpy/lib/nanfunctions.py#L862-L957)). Feel free to check out their complete docstrings.


Your last task is to provide docstrings for the 4 methods you've implemented. Simply go back to those cells and modify your functions. Feel free to use the text provided to you (in the assignment descriptions) to enrich your docstrings. Keep in mind that your docstring needs (1) a general description, (2) a short description for each input, and (3) a short description for the output.

How to check your docstring? Hit `shift+Tab` and see if the pop-up message is correctly compiled, and make sure your description answers all the questions about your functions.